In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('train_news.csv')
df

,Unnamed: 0,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1
...,...,...,...,...,...,...
20795,20795,5671,NaN,NeverSurrender,"No, you'll be a dog licking of the vomit of yo...",1
20796,20796,14831,Albert Pike and the European Migrant Crisis,Rixon Stewart,"By Rixon Stewart on November 5, 2016 Rixon Ste...",1
20797,20797,18142,Dakota Access Caught Infiltrating Protests to ...,Eddy Lavine,posted by Eddie You know the Dakota Access Pip...,1
20798,20798,12139,How to Stretch the Summer Solstice - The New Y...,Alison S. Cohn,"It’s officially summer, and the Society Boutiq...",0


In [2]:
df_testing=df.tail(20)
for i in range(20799,20779,-1):
    df.drop([i],axis=0,inplace=True)
df.shape

(20780, 6)

In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'headline', 'written_by', 'news', 'label'], dtype='object')

In [4]:
data=df.drop(['Unnamed: 0','id','headline','written_by'],axis=1)
#dropping unwanted columns from the dataset
data

,news,label
0,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,HOUSTON — Venezuela had a plan. It was a ta...,0
2,"Sunday on ABC’s “This Week,” while discussing ...",0
3,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,Finian Cunningham has written extensively on...,1
...,...,...
20775,"LUMBERTON, N. C. — Brittany Graham had been...",0
20776,Italian banks struggling to stay afloat Thu Oc...,1
20777,Huckabee Responds To Flag-Burning Trump Hater…...,1
20778,Photo by -ted | CC BY 2.0 \nWhile the recent p...,1


In [5]:
data.isnull().sum()

news     39
label     0
dtype: int64

In [6]:
df_testing.isnull().sum()

Unnamed: 0    0
id            0
headline      1
written_by    2
news          0
label         0
dtype: int64

In [7]:
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)
data.columns

Index(['news', 'label'], dtype='object')

In [8]:
#defining a text processing function
import string
import re
def text_processing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text
data['news']=data['news'].fillna('').apply(text_processing)
x=data['news']
y=data['label']

In [9]:
data.dtypes

news     object
label     int64
dtype: object

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
vector=TfidfVectorizer()
X_train=vector.fit_transform(x_train)
X_test=vector.transform(x_test)

In [11]:
#svc with linear kernel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
clf=SVC(kernel='linear',C=1.0)
clf.fit(X_train,y_train)
pred_svc=clf.predict(X_test)
acc=accuracy_score(y_test,pred_svc)
clf_report=classification_report(y_test,pred_svc)
print("Accuracy obtained by SVC is ",acc)
print("classification report ",clf_report)


Accuracy obtained by SVC is  0.9607314725697786
classification report                precision    recall  f1-score   support

           0       0.96      0.96      0.96      2550
           1       0.96      0.96      0.96      2645

    accuracy                           0.96      5195
   macro avg       0.96      0.96      0.96      5195
weighted avg       0.96      0.96      0.96      5195



In [12]:
map_label={0:'fake',1:'not fake'}  
pred=[map_label[p] for p in pred_svc]
data_test=x_test.copy().reset_index()
data_test['bin_pred']=pred
print(data_test[['news','bin_pred']])

                                                   news  bin_pred
0     when donald j  trump said last fall that he wo...      fake
1     elton john has cancelled all of his upcoming c...      fake
2     assad says the  boy in the ambulance  is fake ...  not fake
3     jackson  miss       thousands of miles away fr...      fake
4     thanksgiving by glen ford   the core ideologic...  not fake
...                                                 ...       ...
5190  oakland  one of california s many sanctuary ci...      fake
5191  yesterday  insofar as anyone can recall  was t...      fake
5192  if you can t give something away change is nee...  not fake
5193  seventeen colombian nationals were arrested by...      fake
5194  report copyright violation mark of the beast  ...  not fake

[5195 rows x 2 columns]


In [ ]:
#svc with poly kernel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
clf=SVC(kernel='poly',C=1.0)
clf.fit(X_train,y_train)
pred_svc=clf.predict(X_test)
acc=accuracy_score(y_test,pred_svc)
clf_report=classification_report(y_test,pred_svc)
print("Accuracy obtained by SVC is ",acc)
print("classification report ",clf_report)


In [ ]:
map_label={0:'fake',1:'not fake'}  
pred=[map_label[p] for p in pred_svc]
data_test=x_test.copy().reset_index()
data_test['bin_pred']=pred
print(data_test[['news','bin_pred']])

In [ ]:
#svc with rbf kernel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
clf=SVC(kernel='rbf',C=1.0)
clf.fit(X_train,y_train)
pred_svc=clf.predict(X_test)
acc=accuracy_score(y_test,pred_svc)
clf_report=classification_report(y_test,pred_svc)
print("Accuracy obtained by SVC is ",acc)
print("classification report ",clf_report)
map_label={0:'fake',1:'not fake'}  
pred=[map_label[p] for p in pred_svc]
data_test=x_test.copy().reset_index()
data_test['bin_pred']=pred
print(data_test[['news','bin_pred']])

In [ ]:
#svc with sigmoid kernel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
clf=SVC(kernel='sigmoid',C=1.0)
clf.fit(X_train,y_train)
pred_svc=clf.predict(X_test)
acc=accuracy_score(y_test,pred_svc)
clf_report=classification_report(y_test,pred_svc)
print("Accuracy obtained by SVC is ",acc)
print("classification report ",clf_report)
map_label={0:'fake',1:'not fake'}  
pred=[map_label[p] for p in pred_svc]
data_test=x_test.copy().reset_index()
data_test['bin_pred']=pred
print(data_test[['news','bin_pred']])